In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import csv
import pickle

In [2]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [3]:
train_data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
train_data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [5]:
train_data['tweet'][1].split()

['@user',
 '@user',
 'thanks',
 'for',
 '#lyft',
 'credit',
 'i',
 "can't",
 'use',
 'cause',
 'they',
 "don't",
 'offer',
 'wheelchair',
 'vans',
 'in',
 'pdx.',
 '#disapointed',
 '#getthanked']

In [8]:
def remove_pattern(input_txt,pattern):
    r=re.findall(pattern,input_txt)
    for i in r:
        input_txt= re.sub(i,'',input_txt)
    return input_txt
    

In [9]:
train_data['tweet']=np.vectorize(remove_pattern)(train_data['tweet'],"@[\w]*")


In [12]:
train_data['tweet'] = train_data['tweet'].str.replace("[^a-zA-Z#]", " ")
train_data.head()

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for #lyft credit i can t use cause th...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide society now #motivation


In [13]:
#removing words less than 4 in length
train_data['tweet']=train_data['tweet'].apply (lambda x: ' '.join(r for r in x.split() if len(r)>3))


In [15]:
tokenized_tweet= train_data['tweet'].apply(lambda x: x.split())

In [17]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])  # stemming
tokenized_tweet.head()

0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, #lyft, credit, caus, they, offer, whee...
2                              [bihday, your, majesti]
3                     [#model, love, take, with, time]
4                         [factsguid, societi, #motiv]
Name: tweet, dtype: object

In [18]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

train_data['tweet'] = tokenized_tweet

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(train_data['tweet'])

In [33]:
bow

<31962x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 124813 stored elements in Compressed Sparse Row format>

In [44]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

clf = SVC(kernel='rbf',random_state=0, gamma=0.51, C=10,probability=True)
#the best params has been found using GridSearchCV

v=clf.fit(xtrain, ytrain)
y_test_predict = clf.predict_proba(xtest)
prediction= y_test_predict[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0

prediction_test = prediction.astype(np.int)
f1_score(ytest, prediction_test)

0.505283381364073

In [47]:
print(v.score(xtest,prediction_test))

0.9957242673897174


In [50]:
print(v.score(xtrain, ytrain))
print(v.score(xtest, ytest))

0.9926250391096411
0.9459797684847221


In [49]:
filename = 'finalized_model.sav'
pickle.dump(v, open(filename, 'wb'))